In [1]:
#r "nuget:Microsoft.ML.LightGBM"
    
//Install XPlot package
#r "nuget:XPlot.Plotly"
#r "nuget:Microsoft.ML"
#r "nuget:Microsoft.ML.DataView"

//Install Daany.DataFrame 
#r "nuget:Daany.DataFrame"
#r "nuget:Daany.DataFrame.Ext"
using System;
using System.Linq;

//Daany data frame
using Daany;
using Daany.Ext;

//Plotting functionalities
using XPlot.Plotly;

//ML.NET using
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Trainers.LightGbm;

Installing package Microsoft.ML.LightGBM..............done!

Successfully added reference to package Microsoft.ML.LightGBM, version 1.4.0

Installing package XPlot.Plotly....done!

Successfully added reference to package XPlot.Plotly, version 3.0.1

Installing package Microsoft.ML....done!

...

Successfully added reference to package Microsoft.ML.DataView, version 1.4.0

Installing package Daany.DataFrame....done!

Successfully added reference to package Daany.DataFrame, version 0.5.3

Installing package Daany.DataFrame.Ext....done!

Successfully added reference to package Daany.DataFrame.Ext, version 0.5.3

# C# Jupyter Notebook Part 2/n

In this blog post, we are going to explore the main features in the new C# Juypter Notebook. For those who used Notebook from other programming languages like Python or R, this would be an easy task. First of all, the Notebook concept provides a quick, simple and straightforward way to present a mix of text and $\LaTeX$, source code implementation and its output. This means you have a full-featured platform to write a paper or blog post, presentation slides, lecture notes, and other educated materials. 

The notebook consists of cells, where a user can write code or markdown text. Once he completes the cell content confirmation for cell editing can be achieved by `Ctrl+Enter` or by press run button from the notebook toolbar. The image below shows the notebook toolbar, with a run button. The popup combo box shows the type of cell the user can define. In the case of text, `Markdown` should be selected, for writing source code the cell should be `Code`.
![run button](img/jupyter_notebook_b2_img01.png)

To start writing code to C# Notebook, the first thing we should do is install NuGet packages or add assembly references and define using statements. In order to do that, the following code installs several `nuget packages`, and declare several `using statements`. But before writing code, we should add a new cell by pressing `+` toolbar button. 

The first few NuGet packages are packages for ML.NET. Then we install the XPlot package for data visualization in .NET Notebook, and then we install a set of Daany packages for data analytics.  First, we install  `Daany.DataFrame` for data exploration and analysis, and then `Daany.DataFrame.Ext` set of extensions for data manipulation used with ML.NET.


Once the NuGet packages are installed successfully, we can start with data exploration. But before this declare few using statements:

We can define classes or methods globally. The following code implements the formatter method for displaying `Daany.DataFrame` in the output cell.


In [20]:
// Temporal DataFrame formatter for this early preview
using Microsoft.AspNetCore.Html;
Formatter<DataFrame>.Register((df, writer) =>
{
    var headers = new List<IHtmlContent>();
    headers.Add(th(i("index")));
    headers.AddRange(df.Columns.Select(c => (IHtmlContent) th(c)));
    
    //renders the rows
    var rows = new List<List<IHtmlContent>>();
    var take = 20;
    
    //
    for (var i = 0; i < Math.Min(take, df.RowCount()); i++)
    {
        var cells = new List<IHtmlContent>();
        cells.Add(td(df.Index[i]));
        foreach (var obj in df[i])
        {
            cells.Add(td(obj));
        }
        rows.Add(cells);
    }
    
    var t = table(
        thead(
            headers),
        tbody(
            rows.Select(
                r => tr(r))));
    
    writer.Write(t);
}, "text/html");

For this demo we will used famous `Iris` data set. We will download the file from the internet, load it by using Daany.DataFrame, a display few first rows. In order to do that we run the folloing code: 

In [21]:
var url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data";
var cols = new string[] {"sepal_length","sepal_width", "petal_length", "petal_width", "flower_type"};
var df = DataFrame.FromWeb(url, sep:',',names:cols);
df.Head(5)

0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5,3.6,1.4,0.2,Iris-setosa


As can be seen, the last line from the previous code has no semicolon, which means the line should be displayed in the output cell. Lets move on, and implement two new columns. Th enew columns will be `sepal` and `petal` area for the flower.
The expression we are going to use is:


$$
PetalArea = petal\_width \cdot petal\_length;\\
SepalArea = sepal\_width \cdot sepal\_length;
$$

As can be seen, the $\LaTeX$ is fully supported in the notebook.

The above formulea is implemented in the following code:

In [4]:
//calculate two new columns into dataset
df.AddCalculatedColumn("SepalArea", (r, i) => Convert.ToSingle(r["sepal_width"]) * Convert.ToSingle(r["sepal_length"]));
df.AddCalculatedColumn("PetalArea", (r, i) => Convert.ToSingle(r["petal_width"]) * Convert.ToSingle(r["petal_length"]));
df.Head(5)

0,5.1,3.5,1.4,0.2,Iris-setosa,17.85,0.28
1,4.9,3,1.4,0.2,Iris-setosa,14.700001,0.28
2,4.7,3.2,1.3,0.2,Iris-setosa,15.04,0.26
3,4.6,3.1,1.5,0.2,Iris-setosa,14.259999,0.3
4,5,3.6,1.4,0.2,Iris-setosa,18,0.28


The data frame has two new columns. They indicate the areas for the flower. In order to see baseic statistics parameters for each of the defined columns we call `Describe` method.

In [5]:
//see descriptive stats of the final ds
df.Describe(false)

Count,150,150,150,150,150,150,150
Unique,35,23,43,22,3,108,101
Top,5,3,1.5,0.2,Iris-setosa,13.200001,0.28
Freq,10,26,14,28,50,5,8
Mean,5.843333,3.054,3.758667,1.198667,<null>,17.806534,5.793133
Std,0.828066,0.433594,1.76442,0.763161,<null>,3.368692,4.713499
Min,4.3,2,1,0.1,<null>,10,0.11
25%,5.1,2.8,1.6,0.3,<null>,15.645,0.42
Median,5.8,3,4.35,1.3,<null>,17.66,5.615
75%,6.4,3.3,5.1,1.8,<null>,20.325001,9.69
Max,7.9,4.4,6.9,2.5,<null>,30.02,15.87


From the table above, we can see the `flower` column has only 3 values. The most frequent value has a frequency equal to 50, which is an indicator for a balanced dataset.

# Data visulaization

The most powerful feature in Notebook is a data visualization. In this section, we are going to plot some interesting charts.

In order to see how sepal and petal areas are spread in 2D plane, the following plot is implemented:


In [6]:
//plot the data in order to see how areas are spread in the 2d plane
//XPlot Histogram reference: http://tpetricek.github.io/XPlot/reference/xplot-plotly-graph-histogram.html

var faresHistogram = Chart.Plot(new Graph.Histogram(){x = df["flower_type"], autobinx = false, nbinsx = 20});
var layout = new Layout.Layout(){title="Distribution of iris flower"};
faresHistogram.WithLayout(layout);
display(faresHistogram);

The chart is also indication of balaced dataset.

Now lets plot areas depending of the flower type:

In [7]:
// Plot Sepal vs. Petal area with flower type

var chart = Chart.Plot(
    new Graph.Scatter()
    {
        x = df["SepalArea"],
        y = df["PetalArea"],
        mode = "markers",
        marker = new Graph.Marker()
        {
            color = df["flower_type"].Select(x=>x.ToString()=="Iris-virginica"?1:(x.ToString()=="Iris-versicolor"?2:3)),
            colorscale = "Jet"
        }
    }
);

var layout = new Layout.Layout(){title="Plot Sepal vs. Petal Area & color scale on flower type"};
chart.WithLayout(layout);
chart.WithLegend(true);
chart.WithLabels(new string[3]{"Iris-virginica","Iris-versicolor", "Iris-setosa"});
chart.WithXTitle("Sepal Area");
chart.WithYTitle("Petal Area");
chart.Width = 800;
chart.Height = 400;

display(chart);

As can be seen from the chart above, flower types are separated almost linearly, since we used petal and sepal areas instead of width and length. With this transformation, we can get a 100% accurate ml model.

# Machine Learning 

Once we finished with data transformation and visualization we can define the final data frame before machine learning application. To end this we are going to select only two columns for features and one label column which will be flower type.

In [8]:
//create new data-frame by selecting only three columns
var derivedDF = df["SepalArea","PetalArea","flower_type"];
derivedDF.Head(5)

0,17.85,0.28,Iris-setosa
1,14.700001,0.28,Iris-setosa
2,15.04,0.26,Iris-setosa
3,14.259999,0.3,Iris-setosa
4,18,0.28,Iris-setosa


Since we are going to use ML.NET, we need to declare `Iris` in order to load the data into ML.NET.

In [9]:
//Define a Iris class for machine learning.
class Iris
{
    public float PetalArea { get; set; }
    public float SepalArea { get; set; }
    public string Species { get; set; }
}
//Create ML COntext
MLContext mlContext = new MLContext(seed:2019);

Then load the data from Daany data frame into ML.NET:

In [10]:
//Load Data Frame into Ml.NET data pipeline
IDataView dataView = mlContext.Data.LoadFromEnumerable<Iris>(derivedDF.GetEnumerator<Iris>((oRow) =>
{
    //convert row object array into Iris row

    var prRow = new Iris();
    prRow.SepalArea = Convert.ToSingle(oRow["SepalArea"]);
    prRow.PetalArea = Convert.ToSingle(oRow["PetalArea"]);
    prRow.Species = Convert.ToString(oRow["flower_type"]);
    //
    return prRow;
}));

Once we have data, we can split it into `train` and `test` sets:

In [11]:
//Split dataset in two parts: TrainingDataset (80%) and TestDataset (20%)
var trainTestData = mlContext.Data.TrainTestSplit(dataView, testFraction: 0.2);
var trainData = trainTestData.TrainSet;
var testData = trainTestData.TestSet;

The next step in prepare the data for training is define pipeline for dtaa transformation and feature engineering:

In [12]:
//one encoding output category column by defining KeyValues for each category
IEstimator<ITransformer> dataPipeline =
mlContext.Transforms.Conversion.MapValueToKey(outputColumnName: "Label", inputColumnName: nameof(Iris.Species))

//define features columns
.Append(mlContext.Transforms.Concatenate("Features",nameof(Iris.SepalArea), nameof(Iris.PetalArea)));

Once we completes the preparation paert, we can perform the training part. The training start by calling `Fit` to the pipeline:

In [13]:
%%time
 // Define LightGbm algorithm estimator
IEstimator<ITransformer> lightGbm = mlContext.MulticlassClassification.Trainers.LightGbm();
//train the ML model
TransformerChain<ITransformer> model = dataPipeline.Append(lightGbm).Fit(trainData);

Wall time: 424.8462ms

Once the training is completes, we have trained model which can be evaluated. In order to print the evaluation result with formatting, we are going to install Daany DataFrame extension which has implementation of printing results 

In [14]:

//evaluate train set
var predictions = model.Transform(trainData);
var metricsTrain = mlContext.MulticlassClassification.Evaluate(predictions);
ConsoleHelper.PrintMultiClassClassificationMetrics("TRAIN Iris DataSet", metricsTrain);
ConsoleHelper.ConsoleWriteHeader("Train Iris DataSet Confusion Matrix ");
ConsoleHelper.ConsolePrintConfusionMatrix(metricsTrain.ConfusionMatrix);

************************************************************
*    Metrics for TRAIN Iris DataSet multi-class classification model   
*-----------------------------------------------------------
    AccuracyMacro = 1, a value between 0 and 1, the closer to 1, the better
    AccuracyMicro = 1, a value between 0 and 1, the closer to 1, the better
    LogLoss = 0.0166, the closer to 0, the better
    LogLoss for class 1 = 0.0074, the closer to 0, the better
    LogLoss for class 2 = 0.0225, the closer to 0, the better
    LogLoss for class 3 = 0.0196, the closer to 0, the better
************************************************************
 
Train Iris DataSet Confusion Matrix 
####################################
 

Confusion table
                   ||========================
PREDICTED          ||     0 |     1 |     2 | Recall
TRUTH              ||========================
0.     Iris-setosa ||    42 |     0 |     0 | 1.0000
1. Iris-versicolor ||     0 |    43 |     0 | 1.0000
2.  Iris-vi

In [15]:
//evaluate test set
var testPrediction = model.Transform(testData);
var metricsTest = mlContext.MulticlassClassification.Evaluate(testPrediction);
ConsoleHelper.PrintMultiClassClassificationMetrics("TEST Iris Dataset", metricsTest);
ConsoleHelper.ConsoleWriteHeader("Test Iris DataSet Confusion Matrix ");
ConsoleHelper.ConsolePrintConfusionMatrix(metricsTest.ConfusionMatrix);

************************************************************
*    Metrics for TEST Iris Dataset multi-class classification model   
*-----------------------------------------------------------
    AccuracyMacro = 1, a value between 0 and 1, the closer to 1, the better
    AccuracyMicro = 1, a value between 0 and 1, the closer to 1, the better
    LogLoss = 0.0102, the closer to 0, the better
    LogLoss for class 1 = 0.0083, the closer to 0, the better
    LogLoss for class 2 = 0.0062, the closer to 0, the better
    LogLoss for class 3 = 0.0172, the closer to 0, the better
************************************************************
 
Test Iris DataSet Confusion Matrix 
###################################
 

Confusion table
                   ||========================
PREDICTED          ||     0 |     1 |     2 | Recall
TRUTH              ||========================
0.     Iris-setosa ||     8 |     0 |     0 | 1.0000
1. Iris-versicolor ||     0 |     7 |     0 | 1.0000
2.  Iris-virgi

As can bee seen, we have 100% accurate model for Iris flower recognition. Now lets add new column into data frame called `Prediction` in order to have model prediction in the data frame.  

In [18]:
var flowerLabels = DataFrameExt.GetLabels(predictions.Schema).ToList();
var p1 = predictions.GetColumn<uint>("PredictedLabel").Select(x=>(int)x).ToList();
var p2 = testPrediction.GetColumn<uint>("PredictedLabel").Select(x => (int)x).ToList();
//join train and test
p1.AddRange(p2);
var p = p1.Select(x => (object)flowerLabels[x-1]).ToList();
//add new column into df
var dic = new Dictionary<string, List<object>> { { "PredictedLabel", p } };

var dff = derivedDF.AddColumns(dic);
dff.Head()

0,17.85,0.28,Iris-setosa,Iris-setosa
1,14.700001,0.28,Iris-setosa,Iris-setosa
2,15.04,0.26,Iris-setosa,Iris-setosa
3,14.259999,0.3,Iris-setosa,Iris-setosa
4,18,0.28,Iris-setosa,Iris-setosa


In [19]:
dff.Tail()

0,20.099998,11.959999,Iris-virginica,Iris-virginica
1,15.75,9.5,Iris-virginica,Iris-virginica
2,19.5,10.4,Iris-virginica,Iris-virginica
3,21.08,12.42,Iris-virginica,Iris-virginica
4,17.7,9.179999,Iris-virginica,Iris-virginica


In this blog post we saw how can we be more productive when using .NET Jupyter Notebook with MAchine Learning and Data Exploration and transformation, by using ML.NET and Daany - **DA**ta**AN**al**Y**tics library.